## Imports, then load the data and inspect it

In [12]:
# Imports
import os
import csv
from datetime import datetime, timedelta
from collections import defaultdict, Counter
from typing import List, Tuple, Dict, Union
from statistics import variance
import unittest

def load_all_data(file_path: str) -> List[Tuple[str, str, datetime]]:
    """Load all event data from a CSV file."""
    all_data = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip header
        for line in csv_reader:
            user_id, event_type, event_time = line
            event_time = datetime.fromisoformat(event_time.replace("Z", "+00:00"))
            all_data.append((user_id, event_type.upper(), event_time))
    return all_data

data = load_all_data('datapao_homework_2023.csv')
data[:3]

[('2e5d8815-4e59-4302-99c0-6fc9593a2eef',
  'GATE_IN',
  datetime.datetime(2023, 1, 31, 8, 18, 36, tzinfo=datetime.timezone.utc)),
 ('a8c60645-aef4-4b4e-aefb-65e242536c2f',
  'GATE_IN',
  datetime.datetime(2023, 1, 31, 8, 43, 41, tzinfo=datetime.timezone.utc)),
 ('0b99d382-ea52-4a1d-8e9e-218933c0d7b8',
  'GATE_IN',
  datetime.datetime(2023, 1, 31, 8, 43, 47, tzinfo=datetime.timezone.utc))]

## 1. Calculate the amount of time and the number of days each person has spent in the office in February. Write results to a CSV, in the format (user_id, time, days, average_per_day, rank)

In [29]:
def filter_and_sort_data(data_source: Union[str, List[Tuple[str, str, datetime]]]) -> List[Tuple[str, str, datetime]]:
    """Filter and sort event data for the month of February."""
    if isinstance(data_source, str):
        all_data = load_all_data(data_source)
        feb_data = [entry for entry in all_data if entry[2].month == 2]
    else:
        feb_data = [entry for entry in data_source if entry[2].month == 2]
    feb_data.sort(key=lambda x: (x[0], x[2]))
    return feb_data

def calculate_user_metrics(feb_data: List[Tuple[str, str, datetime]]) -> Tuple[Dict[str, float], Dict[str, set]]:
    """Calculate metrics like time spent, unique days for each user based on the correct solution."""
    users = defaultdict(lambda: {'time': 0, 'days': set()})

    for user_id, event_type, event_time in feb_data:
        if event_type == 'GATE_IN':
            users[user_id]['current_in'] = event_time
        elif event_type == 'GATE_OUT' and 'current_in' in users[user_id]:
            duration = (event_time - users[user_id]['current_in']).seconds / 3600  # Convert to hours
            users[user_id]['time'] += duration
            users[user_id]['days'].add(event_time.date())
            del users[user_id]['current_in']  # Reset for next pair

    # Transforming the data to align with the previous structure
    user_time = {user: users[user]['time'] for user in users}
    user_days = {user: users[user]['days'] for user in users}

    return user_time, user_days

def calculate_avg_time_and_rank(user_time: Dict[str, float], user_days: Dict[str, set]) -> Tuple[Dict[str, float], Dict[str, int]]:
    """Calculate average time and ordinal rank for each user."""
    user_avg_time = {user: user_time[user] / len(user_days[user]) for user in user_time}
    sorted_users = sorted(user_avg_time.items(), key=lambda x: x[1], reverse=True)
    user_rank = {user: rank + 1 for rank, (user, _) in enumerate(sorted_users)}
    return user_avg_time, user_rank

def write_to_csv(output_data: List[List[Union[str, int, float]]], header: List[str], file_path: str) -> None:
    """Write calculated metrics to a CSV file."""
    with open(file_path, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(header)
        csv_writer.writerows(output_data)

def task_one_main(file_path: str, output_file_path: str) -> None:
    """Main function to execute Task 1: Calculate and write user metrics to a CSV."""
    feb_data = filter_and_sort_data(file_path)
    user_time, user_days = calculate_user_metrics(feb_data)
    user_avg_time, user_rank = calculate_avg_time_and_rank(user_time, user_days)

    output_data = [[user, user_time[user], len(user_days[user]), user_avg_time[user], user_rank[user]] for user in user_time]

    # Ensure the output directory exists
    output_directory = 'output/'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Define the complete output file path
    complete_output_file_path = os.path.join(output_directory, output_file_path)
    print(complete_output_file_path)

    # Write the data to a CSV file
    write_to_csv(output_data, ['user_id', 'time', 'days', 'average_per_day', 'rank'], complete_output_file_path)


### Some test driver code
# # Recalculate user metrics using the corrected function
# user_time_corrected, user_days_corrected = calculate_user_metrics(filter_and_sort_data('datapao_homework_2023.csv'))
# 
# # Recalculate average time per day and rank
# user_avg_time_corrected, user_rank_corrected = calculate_avg_time_and_rank(user_time_corrected, user_days_corrected)
# 
# # Create the output for the first part of the exercise
# output_first_corrected = [
#     [user, user_time_corrected[user], len(user_days_corrected[user]), user_avg_time_corrected[user], user_rank_corrected[user]]
#     for user in user_time_corrected
# ]
# 
# # Sort by rank to show top 5
# output_first_corrected.sort(key=lambda x: x[4])

# Call the main function to execute the first task and write the results to a CSV file
task_one_main('datapao_homework_2023.csv', 'first.csv')

output/first.csv


In [30]:
# Unit Tests for Task 1

class TestTaskOne(unittest.TestCase):

    # Sample data for testing
    sample_data = [
        ('user_1', 'GATE_IN', datetime(2023, 2, 1, 8, 0, 0)),
        ('user_1', 'GATE_OUT', datetime(2023, 2, 1, 12, 0, 0)),
        ('user_1', 'GATE_IN', datetime(2023, 2, 1, 13, 0, 0)),
        ('user_1', 'GATE_OUT', datetime(2023, 2, 1, 18, 0, 0)),
        ('user_2', 'GATE_IN', datetime(2023, 2, 1, 9, 0, 0)),
        ('user_2', 'GATE_OUT', datetime(2023, 2, 1, 17, 0, 0))
    ]

    def test_filter_and_sort_data(self):
        self.assertEqual(filter_and_sort_data(self.sample_data), self.sample_data)

    def test_calculate_user_metrics(self):
        user_time, user_days = calculate_user_metrics(self.sample_data)
        self.assertEqual(user_time['user_1'], 9.0)  # Changed
        self.assertEqual(user_time['user_2'], 8.0)  # Changed
        self.assertEqual(len(user_days['user_1']), 1)
        self.assertEqual(len(user_days['user_2']), 1)

    def test_calculate_avg_time_and_rank(self):
        user_time = {'user_1': 9.0, 'user_2': 8.0}  # Changed
        user_days = {'user_1': {datetime(2023, 2, 1).date()}, 'user_2': {datetime(2023, 2, 1).date()}}
        user_avg_time, user_rank = calculate_avg_time_and_rank(user_time, user_days)
        self.assertEqual(user_avg_time['user_1'], 9.0)
        self.assertEqual(user_avg_time['user_2'], 8.0)
        self.assertEqual(user_rank['user_1'], 1)
        self.assertEqual(user_rank['user_2'], 2)

# Run the unit tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestTaskOne))

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## 2. Find out who had the longest work session in February. Write the result to a CSV in the format (user_id, session_length)

In [33]:
def calculate_user_sessions(feb_data: List[Tuple[str, str, datetime]]) -> Dict[str, List[Tuple[datetime, datetime]]]:
    """Calculate metrics like time spent, unique days, and sessions for each user."""
    user_sessions = defaultdict(list)

    for i in range(len(feb_data) - 1):
        user_id, event_type, event_time = feb_data[i]
        next_user_id, next_event_type, next_event_time = feb_data[i + 1]

        if user_id == next_user_id and event_type == "GATE_IN" and next_event_type == "GATE_OUT":
            if user_sessions[user_id]:
                last_session_end = user_sessions[user_id][-1][1]
                if (event_time - last_session_end).seconds // 3600 >= 2:
                    user_sessions[user_id].append((event_time, next_event_time))
                else:
                    user_sessions[user_id][-1] = (user_sessions[user_id][-1][0], next_event_time)
            else:
                user_sessions[user_id].append((event_time, next_event_time))

    return user_sessions

def calculate_longest_session(user_sessions: Dict[str, List[Tuple[datetime, datetime]]]) -> Dict[str, timedelta]:
    """Calculate the longest work session for each user."""
    return {user: max((session[1] - session[0] for session in sessions), default=timedelta(0))
            for user, sessions in user_sessions.items()}

def task_two_main(file_path: str, output_file_path: str) -> None:
    """Main function to execute Task 2: Calculate and write the longest work sessions to a CSV."""
    feb_data = filter_and_sort_data(file_path)
    user_sessions = calculate_user_sessions(feb_data)

    user_longest_session = calculate_longest_session(user_sessions)
    sorted_users_by_session = sorted(user_longest_session.items(), key=lambda x: x[1], reverse=True)
    output_second = [[user, longest_session.seconds // 3600] for user, longest_session in sorted_users_by_session]

    # Ensure the output directory exists
    output_directory = 'output/'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Define the complete output file path
    complete_output_file_path = os.path.join(output_directory, output_file_path)
    print(complete_output_file_path)

    # Write the data to a CSV file
    write_to_csv(output_second, ['user_id', 'session_length'], complete_output_file_path)

# Define output file path for Task 2
output_file_path_2 = 'second.csv'

# Call the main function for Task 2
task_two_main('datapao_homework_2023.csv', output_file_path_2)

output/second.csv


In [34]:
# Unit Tests for Task 2

class TestTaskTwo(unittest.TestCase):

    # Sample session data for testing
    sample_sessions = {
        'user_1': [
            (datetime(2023, 2, 1, 8, 0, 0), datetime(2023, 2, 1, 18, 0, 0)),  # Merged session
        ],
        'user_2': [
            (datetime(2023, 2, 1, 9, 0, 0), datetime(2023, 2, 1, 17, 0, 0))
        ]
    }

    def test_calculate_longest_session(self):
        longest_sessions = calculate_longest_session(self.sample_sessions)
        self.assertEqual(longest_sessions['user_1'].seconds // 3600, 10)
        self.assertEqual(longest_sessions['user_2'].seconds // 3600, 8)

# Run the unit tests for Task 2
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestTaskTwo))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

## 3.1 Identify core working hours -- when are the most employees present?

This should help us best book face-to-face meetings and collaborative work.

In [36]:
def calculate_hourly_presence(feb_data: List[Tuple[str, str, datetime]]) -> Dict[int, int]:
    """Calculate the hourly presence of employees."""
    hourly_presence = Counter()
    for user_id, event_type, event_time in feb_data:
        if event_type == "GATE_IN":
            hourly_presence[event_time.hour] += 1
        elif event_type == "GATE_OUT":
            hourly_presence[event_time.hour] -= 1
    return hourly_presence

def calculate_cumulative_hourly_presence(hourly_presence: Dict[int, int]) -> Dict[int, int]:
    """Calculate the cumulative hourly presence of employees."""
    cumulative_presence = 0
    hourly_cumulative_presence = {}
    for hour in range(24):
        cumulative_presence += hourly_presence.get(hour, 0)
        hourly_cumulative_presence[hour] = cumulative_presence
    return hourly_cumulative_presence

# Run the functions for calculating core working hours
hourly_presence = calculate_hourly_presence(filter_and_sort_data('datapao_homework_2023.csv'))
hourly_cumulative_presence = calculate_cumulative_hourly_presence(hourly_presence)
hourly_cumulative_presence

{0: -1,
 1: -2,
 2: -3,
 3: -3,
 4: -3,
 5: -3,
 6: -3,
 7: -3,
 8: 57,
 9: 131,
 10: 153,
 11: 189,
 12: 176,
 13: 180,
 14: 175,
 15: 179,
 16: 174,
 17: 173,
 18: 163,
 19: 137,
 20: 56,
 21: -3,
 22: -2,
 23: 0}

In [37]:
# Unit Tests for Core Working Hours Calculation

class TestCoreWorkingHours(unittest.TestCase):

    # Sample data for testing
    sample_data = [
        ('user_1', 'GATE_IN', datetime(2023, 2, 1, 8, 0, 0)),
        ('user_1', 'GATE_OUT', datetime(2023, 2, 1, 12, 0, 0)),
        ('user_1', 'GATE_IN', datetime(2023, 2, 1, 13, 0, 0)),
        ('user_1', 'GATE_OUT', datetime(2023, 2, 1, 18, 0, 0)),
        ('user_2', 'GATE_IN', datetime(2023, 2, 1, 9, 0, 0)),
        ('user_2', 'GATE_OUT', datetime(2023, 2, 1, 17, 0, 0))
    ]

    def test_calculate_hourly_presence(self):
        hourly_presence = calculate_hourly_presence(self.sample_data)
        self.assertEqual(hourly_presence[8], 1)
        self.assertEqual(hourly_presence[9], 1)
        self.assertEqual(hourly_presence[12], -1)
        self.assertEqual(hourly_presence[13], 1)
        self.assertEqual(hourly_presence[17], -1)
        self.assertEqual(hourly_presence[18], -1)

    def test_calculate_cumulative_hourly_presence(self):
        hourly_presence = calculate_hourly_presence(self.sample_data)
        hourly_cumulative_presence = calculate_cumulative_hourly_presence(hourly_presence)
        self.assertEqual(hourly_cumulative_presence[8], 1)
        self.assertEqual(hourly_cumulative_presence[9], 2)
        self.assertEqual(hourly_cumulative_presence[12], 1)
        self.assertEqual(hourly_cumulative_presence[13], 2)
        self.assertEqual(hourly_cumulative_presence[17], 1)
        self.assertEqual(hourly_cumulative_presence[18], 0)

# Run the unit tests for Core Working Hours Calculation
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestCoreWorkingHours))

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

At 11 AM we have the highest employee presence, and it might be best to book larger meetings around that time.

The number of people present remains somewhat constant for multiple hours after that, so these hours might be best for any collaborative work sessions.

## 3.2 Flexibility index

Calculate the variance in the start and end times of each employee's work sessions. A higher variance indicates the employee has a very flexible schedule, and a lower variance indicates the employee has a strict pattern and routine.

The most flexible workers might be our first people to call when urgent things pop up outside more typical working hours.
The least flexible users are predictable, and their schedules might be the easiest to plan around and assign tasks based on.

In [39]:
def populate_start_end_times(user_sessions: Dict[str, List[Tuple[datetime, datetime]]]) -> Tuple[Dict[str, List[int]], Dict[str, List[int]]]:
    """Populate start and end times for each user's work sessions."""
    user_start_times = defaultdict(list)
    user_end_times = defaultdict(list)
    for user, sessions in user_sessions.items():
        for session_start, session_end in sessions:
            user_start_times[user].append(session_start.hour)
            user_end_times[user].append(session_end.hour)
    return user_start_times, user_end_times

def calculate_flexibility_index(user_start_times: Dict[str, List[int]], user_end_times: Dict[str, List[int]]) -> Dict[str, float]:
    """Calculate the flexibility index for each user."""
    user_flexibility_index = {}
    for user in user_start_times:
        start_var = variance(user_start_times[user]) if len(user_start_times[user]) > 1 else 0
        end_var = variance(user_end_times[user]) if len(user_end_times[user]) > 1 else 0
        user_flexibility_index[user] = start_var + end_var
    return user_flexibility_index

# Run the functions for calculating flexibility index
user_start_times, user_end_times = populate_start_end_times(calculate_user_sessions(filter_and_sort_data('datapao_homework_2023.csv')))
user_flexibility_index = calculate_flexibility_index(user_start_times, user_end_times)

# Sort the users by Flexibility Index
sorted_users_by_flexibility = sorted(user_flexibility_index.items(), key=lambda x: x[1], reverse=True)

# Display the top 3 most flexible and least flexible users
top_3_most_flexible_users = sorted_users_by_flexibility[:3]
top_3_least_flexible_users = sorted_users_by_flexibility[-3:]

top_3_most_flexible_users, top_3_least_flexible_users

([('1fcdd2f5-13ed-4ee8-b4ec-706654e68472', 57.82213438735178),
  ('bfa0f1f9-300e-41ad-bcff-0c3bd79595f1', 40.01307189542484),
  ('5dafb3b2-22aa-4039-a2f0-3cf711f84177', 33.97011494252874)],
 [('98ea63ee-bba1-4ea5-8ada-1d0a2dd1f6fd', 23.04),
  ('oab91046-1831-436d-a41d-da6ba1b2d385', 22.104761904761904),
  ('33c08c48-f50a-4c72-a975-f8572d65a8db', 22.026666666666664)])

In [40]:
# Unit Tests for Flexibility Index Calculation

class TestFlexibilityIndex(unittest.TestCase):

    # Sample session data for testing
    sample_sessions = {
        'user_1': [
            (datetime(2023, 2, 1, 8, 0, 0), datetime(2023, 2, 1, 18, 0, 0)),  # Merged session
            (datetime(2023, 2, 2, 9, 0, 0), datetime(2023, 2, 2, 17, 0, 0))
        ],
        'user_2': [
            (datetime(2023, 2, 1, 9, 0, 0), datetime(2023, 2, 1, 17, 0, 0)),
            (datetime(2023, 2, 2, 10, 0, 0), datetime(2023, 2, 2, 16, 0, 0))
        ]
    }

    def test_populate_start_end_times(self):
        start_times, end_times = populate_start_end_times(self.sample_sessions)
        self.assertEqual(start_times['user_1'], [8, 9])
        self.assertEqual(end_times['user_1'], [18, 17])
        self.assertEqual(start_times['user_2'], [9, 10])
        self.assertEqual(end_times['user_2'], [17, 16])

    def test_calculate_flexibility_index(self):
        start_times, end_times = populate_start_end_times(self.sample_sessions)
        flexibility_index = calculate_flexibility_index(start_times, end_times)
        self.assertEqual(flexibility_index['user_1'], 0.5 + 0.5)
        self.assertEqual(flexibility_index['user_2'], 0.5 + 0.5)

# Run the unit tests for Flexibility Index Calculation
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestFlexibilityIndex))

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

## Solving the task with pandas to get a "correct" solution for debugging purposes

In [1]:
import pandas as pd

df = pd.read_csv('datapao_homework_2023.csv')

df.head()

,user_id,event_type,event_time
0,2e5d8815-4e59-4302-99c0-6fc9593a2eef,GATE_IN,2023-01-31T08:18:36.000Z
1,a8c60645-aef4-4b4e-aefb-65e242536c2f,GATE_IN,2023-01-31T08:43:41.000Z
2,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,gate_in,2023-01-31T08:43:47.000Z
3,d22c03ba-00f7-4473-bc9d-61136643994f,GATE_IN,2023-01-31T09:04:08.000Z
4,1fcdd2f5-13ed-4ee8-b4ec-706654e68472,GATE_IN,2023-01-31T09:12:56.000Z


In [2]:
df['event_time'] = pd.to_datetime(df['event_time'])
df['event_type'] = df['event_type'].str.upper()

df_feb = df[(df['event_time'].dt.month == 2) & (df['event_time'].dt.year == 2023)]

df_feb = df_feb.sort_values(by=['user_id', 'event_time'])

df_feb.head()

,user_id,event_type,event_time
98,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,GATE_IN,2023-02-01 08:29:47+00:00
120,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,GATE_OUT,2023-02-01 11:49:51+00:00
132,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,GATE_IN,2023-02-01 13:22:48+00:00
144,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,GATE_OUT,2023-02-01 14:36:19+00:00
168,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,GATE_IN,2023-02-01 18:27:42+00:00


In [8]:
from collections import defaultdict
import numpy as np

# Initialize an empty list to store individual DataFrames
dfs_to_concat = []

# Group by user and then iterate over each group to calculate time spent in the office, number of days, and average time per day
for user, user_data in df_feb.groupby('user_id'):
    user_data = user_data.sort_values('event_time')
    time_spent = 0
    days = set()
    last_gate_in = None

    for i, row in user_data.iterrows():
        if row['event_type'] == 'GATE_IN':
            last_gate_in = row['event_time']
        elif row['event_type'] == 'GATE_OUT' and last_gate_in is not None:
            time_spent += (row['event_time'] - last_gate_in).seconds / 3600  # Convert to hours
            last_gate_in = None  # Reset for the next pair

        days.add(row['event_time'].date())

    avg_time_per_day = time_spent / len(days)

    # Create a DataFrame for this user and add to the list
    user_df = pd.DataFrame({
        'user_id': [user],
        'time': [time_spent],
        'days': [len(days)],
        'average_per_day': [avg_time_per_day]
    })
    dfs_to_concat.append(user_df)

# Use `pd.concat` to concatenate all the individual DataFrames
results = pd.concat(dfs_to_concat, ignore_index=True)

# Calculate the rank based on average_per_day
results['rank'] = results['average_per_day'].rank(method='min').astype(int)

# Sort the DataFrame by 'average_per_day' in descending order and then calculate the rank
results = results.sort_values(by='average_per_day', ascending=False)
results['rank'] = results['average_per_day'].rank(method='min', ascending=False).astype(int)

# Display the corrected DataFrame sorted by the new rank
results

,user_id,time,days,average_per_day,rank
20,d22c03ba-00f7-4473-bc9d-61136643994f,71.219722,11,6.474520,1
16,a8c60645-aef4-4b4e-aefb-65e242536c2f,107.857500,17,6.344559,2
24,tc83470b-61c3-48c5-9bf3-9d623942e2ec,95.784722,16,5.986545,3
17,abce78a1-f8e6-4e9b-aafe-9786846e089e,79.129444,14,5.652103,4
23,p3b4e81f-79de-4937-ba87-aec8e7e731af,105.220833,19,5.537939,5
15,a59f9f64-2937-40e6-bc28-e43fbab63a65,75.745278,14,5.410377,6
22,oab91046-1831-436d-a41d-da6ba1b2d385,59.390278,11,5.399116,7
4,2c4c1aa0-5dd6-4fc1-b474-6c2ed92dd43c,96.889444,18,5.382747,8
0,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,89.756667,17,5.279804,9
5,2e5d8815-4e59-4302-99c0-6fc9593a2eef,82.947778,16,5.184236,10


In [9]:
# To calculate the longest work session, we'll consider a gap of two or more hours between an 'OUT' and the next 'IN' as the end of a session.
# We'll group the data by user, then iterate over each user's data to identify sessions and their lengths.

# Initialize an empty list to store the results
longest_sessions = []

# Define the threshold for a new session in hours
threshold = 2

# Group by user and then iterate over each group to identify work sessions
for user, user_data in df_feb.groupby('user_id'):
    user_data = user_data.sort_values('event_time')
    longest_session = 0
    session_start = None
    last_event_time = None
    last_event_type = None

    for i, row in user_data.iterrows():
        if row['event_type'] == 'GATE_IN':
            # Check if the gap between the last 'OUT' and this 'IN' is greater than the threshold
            if last_event_type == 'GATE_OUT' and (row['event_time'] - last_event_time).seconds / 3600 >= threshold:
                # End the current session and start a new one
                session_length = (last_event_time - session_start).seconds / 3600  # Convert to hours
                longest_session = max(longest_session, session_length)
                session_start = row['event_time']
            elif session_start is None:
                session_start = row['event_time']

        elif row['event_type'] == 'GATE_OUT':
            if session_start is not None:
                session_length = (row['event_time'] - session_start).seconds / 3600  # Convert to hours
                longest_session = max(longest_session, session_length)

        last_event_time = row['event_time']
        last_event_type = row['event_type']

    # Append the result to the list
    longest_sessions.append({
        'user_id': user,
        'session_length': longest_session
    })

# Create a DataFrame from the list
df_longest_sessions = pd.DataFrame(longest_sessions)

# Sort the DataFrame by session_length in descending order to identify the user with the longest work session
df_longest_sessions = df_longest_sessions.sort_values(by='session_length', ascending=False)

df_longest_sessions

,user_id,session_length
23,p3b4e81f-79de-4937-ba87-aec8e7e731af,12.770556
6,33c08c48-f50a-4c72-a975-f8572d65a8db,12.768611
24,tc83470b-61c3-48c5-9bf3-9d623942e2ec,12.705556
16,a8c60645-aef4-4b4e-aefb-65e242536c2f,12.430278
10,5dafb3b2-22aa-4039-a2f0-3cf711f84177,12.067500
8,45887b17-5c63-410d-9ae3-e645743c3076,12.003611
4,2c4c1aa0-5dd6-4fc1-b474-6c2ed92dd43c,11.358333
5,2e5d8815-4e59-4302-99c0-6fc9593a2eef,11.329722
13,8ee31b32-9b50-47da-8927-801a62348cb5,10.999444
0,0b99d382-ea52-4a1d-8e9e-218933c0d7b8,10.961111
